In [ ]:
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from scipy.ndimage  import gaussian_filter1d

import json

In [ ]:
from masking import *
from latencies import *
from excitation import *
from deconv import *
from ur import *
from tuning import *

### Simulate CAPs

In [ ]:
#masking degree function
mdFunc=SigmoidMaskingDegreeFunction(30, 2*1/15., requires_grad=False)

#latencies
lat0=Eggermont1976clickLatencies80dB #latencies

#Unitary response
ur0=URWang1979m
t=np.linspace(5e-4, 10e-3, num=500)
u00=ur0.u(t-2e-3) #shift
sig=0.2e-3 #std of gaussian kernel in s
sig2=sig/(t[1]-t[0])
u0 = gaussian_filter1d(u00, sigma=sig2)
u0/=np.amax(np.abs(u0))
u0*=0.5*np.amax(np.abs(u00))

#tuning
BW10_0=500
BW10Func=constant_BW10(BW10_0, requires_grad=False)

plotMaskingDegreeFunc(mdFunc)
plotLatencies(lat0)

In [ ]:
pl.plot(t*1e3, u00, label='unitary response')
pl.plot(t*1e3, u0, label='ur + nPSHT (blur)',  color='orange')
pl.legend(loc='lower right')
pl.title("Unitary response")
pl.xlabel('t (ms)')

Example: one masker

In [ ]:
json_txt='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":3, 
    "bands":[
        {
            "amplitude":0.02512, 
            "fc_low":2662.5, 
            "fc_high":4162.5
        }, 
        {
            "amplitude":0.1, 
            "fc_low":4162.5, 
            "fc_high":5375
        }, 
        {
            "amplitude":0.01585, 
            "fc_low":5375, 
            "fc_high":6808.5
        }
    ], 
    "name":"37-2bands-4769-1213-3412-1500-32dB"
}'''

stim_struct=json.loads(json_txt)

In [ ]:
def 